In [1]:
import ast
import re
import string
import data_utils

In [1]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['real_estate_data']
read_collection = db['real_estate_query_data']
write_collection = db['real_estate_query_data_normailze']

In [16]:
count = 0
for document in read_collection.find():
#     document["price"] = normalize_price(document["price"])
    print(document["price"])
#     write_collection.insert_one(document)
    count = count+1
    if count > 1000:
        break

: 2 tỷ 900 triệu
: 3 tỷ 950 triệu
: 3 tỷ 550 triệu
: 2 tỷ 300 triệu
: 3 tỷ 800 triệu
: 30 triệu
: 3 tỷ 500 triệu
: 800 triệu
: 5 tỷ 450 triệu
: 1 tỷ 250 triệu
: 2 tỷ 600 triệu
: 4 tỷ 788 triệu
: 1 tỷ 845 triệu
: 3 tỷ 80 triệu
: 330 triệu
: 4 tỷ 700 triệu
: 900 triệu
: 900 triệu
: 5 tỷ 300 triệu
: 1 tỷ 750 triệu
: 3 tỷ 450 triệu
: 5 tỷ 500 triệu
: 3 tỷ 500 triệu
: 2 tỷ 800 triệu
: 1 tỷ 750 triệu
: 2 tỷ 488 triệu
: 1 tỷ 360 triệu
: 3 tỷ 50 triệu
: 3 tỷ 100 triệu
: 1 tỷ 700 triệu
: 1 tỷ 550 triệu
: 3 tỷ 150 triệu
: 2 tỷ 300 triệu
: 1 tỷ 350 triệu
: 1 tỷ 350 triệu
: 1 tỷ 350 triệu
: 1 tỷ 270 triệu
: 988 triệu
: 900 triệu
: 1 tỷ
: 3 tỷ 900 triệu
: 1 tỷ 200 triệu
: 1 tỷ 400 triệu
: 1 tỷ 455 triệu
: 1 tỷ 389 triệu
: 1 tỷ 200 triệu
: 900 triệu
: 1 tỷ 400 triệu
: 1 tỷ 900 triệu
: 860 triệu
: 900 triệu
: 200 triệu
: 3 tỷ 900 triệu
: 2 tỷ 250 triệu
: 3 tỷ 900 triệu
: 1 tỷ 100 triệu
: 3 tỷ 900 triệu
: 2 tỷ
: 1 tỷ 260 triệu
: 2 tỷ 720 triệu
: 2 tỷ 200 triệu
: 1 tỷ 650 triệu
: 2 tỷ
: 1 tỷ 400 triệu
